## Implementation of Visualizing Data Using t-SNE

#### Aasha Reddy and Madeleine Beckner

#### Due: April 27, 2021

### Abstract

250 words or less. Identify 4-6 key phrases.

t-SNE. In this report, we will implement the t-SNE algorithm as described in the paper Visualizing Data using t-SNE (Hinton and van der Maaten, 2008). We will demonstrate its accuracy and performance through various applications to both real and simulated datasets, optimize the code, and compare this algorithm with two other competing algorithms that address a similar problem.

Key phrases: t-SNE, stochastic neighbor embedding, dimensionality reduction, visualization

### Background

State the research paper you are using. Describe the concept of the algorithm and why it is interesting and/or useful. If appropriate, describe the mathematical basis of the algorithm. Some potential topics for the background include:
What problem does it address?
What are known and possible applications of the algorithm?
What are its advantages and disadvantages relative to other algorithms?
How will you use it in your research?

In this paper, we will implement the t-SNE algorithm for visualizing high-dimensional data as described in the paper Visualizing Data using t-SNE (Hinton and van der Maaten, 2008). This algorithm gives each datapoint a corresponding location in a two or three-dimensional mapping. The t-SNE algorithm improves upon the Stochastic Neighbor Embedding algorithm (Hinton and Roweis, 2002) by reducing issues with points crowding in the center of the map.

Visualizing high-dimensional data in an effective and interpretable way is important to many different areas of application. High-dimensional datasets are common in numerous fields of study, such as biology, chemistry, political science, economics, astronomy, physics, and more. As availability of computing resources continues to increase, so does the ability to collect and store more and more complex and high-dimensional datasets. Effective visualization  presents an interesting and important challenge in analyzing and understanding these complex datasets.

May different high-dimensional data methods have been proposed in recent years, but many of these methods come with certain issues. For example, techniques that use iconographic displays such as pixel-based techniques or Chernoff faces provide tools to visualize high-dimensional data while leaving the interpretation to the viewer. When scaled to thousands of dimensions, as is often the case with real-world data, these methods break down due to human inabi. (1) it is unclear how t-SNE performs on general dimensionality reduction tasks, (2) the relatively local nature of t-SNE makes it sensitive to the curse of the intrinsic dimensionality of the data, and (3) t-SNE is not guaranteed to converge to a global optimum of its cost function. Below, we discuss the three weaknesses in more detail.ity to interpret this kind of visual dimension scale. To avoid this issue, methods with dimensionality-reduction have been proposed that convert high-dimensional data into two or three dimensions to be plotted. Popular linear dimensionality reduction methods include PCA, or Principal Components Analysis, and MDS, Multi-Dimensional Scaling. However, there can be issues here in applications where low-dimensional representations of similar points must be kept close together, because these methods prioritize keeping low-dimensional representations of dissimilar points far apart. To avoid this issue and preserve local data structures, non-linear dimensionality reduction methods have been proposed as well, one of which, Stochastic Neighbor Embedding, forms the basis of this paper. While these types of methods improve over other mentioned concerns and generally perform well on artificial datasets, they tend to break down when introduced to real-world high dimensional datasets because they are not capable of retaining both local and the global structure of the data in a single map. The method t-SNE presented in this paper provides a solution by being capable of capturing both local and global structures effectively. t-SNE, therefore, has many advantages relative to other algorithms. 

While t-SNE presents improvements from other existing methods, this algorithm has its own weaknesses as well. First, it is not clear how t-SNE performs on general dimensionality reduction tasks where d > 3 due to the heavy tails of the Student-t distribution. Additionally, the relatively local nature of t-SNE makes it sensitive to the curse of the intrinsic dimensionality of the data, which means t-SNE might be less successful if it is applied on data sets with a very high intrinsic dimensionality. The most significant potential weakness is that t-SNE is not guaranteed to converge to a global optimum of its cost function due to the non-convexity of the cost function. Therefore, we have to choose several optimization parameters, and the solutions may differ depending on the choice of these parameters. In spite of these drawbacks, t-SNE still outperforms other similar methods and improves upon common issues in high-dimensional visualization methods. In our paper, we will implement and evaluate the t-SNE metric on multiple datasets. 






### Description of Algorithm

First, explain in plain English what the algorithm does. Then describes the details of the algorithm, using mathematical equations or pseudocode as appropriate.



STOchastic then t-sne

### Describe Optimization for Performance

First implement the algorithm using plain Python in a straightforward way from the description of the algorithm. Then profile and optimize it using one or more appropriate methods, such as:

1. Use of better algorithms or data structures
2. Use of vectorization
3. JIT or AOT compilation of critical functions
4. Re-writing critical functions in C++ and using pybind11 to wrap them
5. Making use of parallelism or concurrency
6. Making use of distributed compuitng

Document the improvement in performance with the optimizations performed.

### Applications to Simulated Datasets

Are there specific inputs that give known outputs (e.g. there might be closed form solutions for special input cases)? How does the algorithm perform on these? 

If no such input cases are available (or in addition to such input cases), how does the algorithm perform on simulated data sets for which you know the "truth"? 

### Applications to Real Datasets

Test the algorithm on the real-world examples in the original paper if possible. Try to find at least one other real-world data set not in the original paper and test it on that. Describe and interpret the results.

### Comparative Analysis to Competing Algorithms

Find two other algorithm that address a similar problem. Perform a comparison - for example, of accuracy or speed. You can use native libraries of the other algorithms - you do not need to code them yourself. Comment on your observations. 

### Discussion and Conclusion

Your thoughts on the algorithm. Does it fulfill a particular need? How could it be generalized to other problem domains? What are its limitations and how could it be improved further?

### References

1. van der Maaten, Laurens and Hinton, Geoffrey. "Visualizing Data using t-SNE ." Journal of Machine Learning Research 9 (2008): 2579--2605.
2. Sci-kit Learn implementation: https://github.com/scikit-learn/scikit-learn/blob/95119c13af77c76e150b753485c662b7c52a41a2/sklearn/manifold/_utils.pyx
http://www.sci.utah.edu/~beiwang/publications/Vis_HD_STAR_BeiWang_2015.pdf

### Code